一，准备工作

1，包的安装

In [ ]:
#1，终端中安装使用pip
pip install 'scanpy[leiden]'
pip install python-igraph


#2，使用conda安装，采用法2
mamba create -n sc-python -c conda-forge -y scanpy python-igraph leidenalg python=3.12
conda activate sc-python

二， AnnData数据结构理解

1，构建模拟Anndata对象并查看

In [1]:
# 导包
import numpy as np
import pandas as pd
import anndata as ad #导入 anndata 库，并将其命名为 ad。anndata 是一个用于处理单细胞基因表达数据的库，提供了 AnnData 对象，用于存储和操作高维数据
from scipy.sparse import csr_matrix #从 scipy.sparse 模块中导入 csr_matrix 类。csr_matrix 是一种压缩稀疏行矩阵格式，用于高效存储和操作稀疏矩阵
print(ad.__version__)

0.11.3


In [2]:
# 模拟一个矩阵构建AnnData对象
counts = csr_matrix(np.random.poisson(1, size=(100, 2000)), dtype=np.float32) #将生成的随机矩阵转换为 scipy.sparse 模块中的压缩稀疏行矩阵（CSR 格式），并指定数据类型为 float32
temp_adata = ad.AnnData(counts) 
print(f"这个AnnData对象包括{temp_adata.shape[0]}行，与{temp_adata.shape[1]}列") #使用 anndata 库的 AnnData 类创建一个 AnnData 对象，并将稀疏矩阵 counts 作为数据存储在其中
# 稀疏矩阵存放在temp_adata.X中

这个AnnData对象包括100行，与2000列


In [7]:
# 生成细胞名称并传递给obs_names
temp_adata.obs_names = [f"Cell_{i:d}" for i in range(temp_adata.n_obs)]
# 生成基因名称并传递给var_names
temp_adata.var_names = [f"Gene_{i:d}" for i in range(temp_adata.n_vars)]

print(f"前五个细胞名称为:\n{temp_adata.obs_names[:5]}")

前五个细胞名称为:
Index(['Cell_0', 'Cell_1', 'Cell_2', 'Cell_3', 'Cell_4'], dtype='object')


In [ ]:
# 那么子集就可以这么取:行列切片
temp_adata[["Cell_1", "Cell_10"], ["Gene_5", "Gene_1900"]]
# 这样就得到了一个包含"两个细胞"、"两个基因"的表达矩阵

View of AnnData object with n_obs × n_vars = 2 × 2

2，添加注释信息

In [19]:
print(type(temp_adata.obs))
print(type(temp_adata.var))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [32]:
# 例如这里随机生成"B", "T", "Monocyte"三种细胞类型作为注释
ct = np.random.choice(["B", "T", "Monocyte"], size=(temp_adata.n_obs,))
print(f"ct的前10个元素为{ct[0:10]}")

# 加入adata.obs中

ct的前10个元素为['B' 'T' 'Monocyte' 'B' 'Monocyte' 'B' 'Monocyte' 'B' 'T' 'Monocyte']


In [41]:
temp_adata.obs["cell_type"] = pd.Categorical(ct)  # Categorical的执行效率较高

# 可以看到adata.obs已经包含了我们刚刚添加的注释
temp_adata.obs

,cell_type
Cell_0,B
Cell_1,T
Cell_2,Monocyte
Cell_3,B
Cell_4,Monocyte
...,...
Cell_95,B
Cell_96,Monocyte
Cell_97,T
Cell_98,B


In [44]:
# 例如这里随机生成"Pathway_A", "Pathway_B", "Pathway_C"三种通路名作为细胞的注释
my_path = np.random.choice(["Pathway_A", "Pathway_B", "Pathway_C"],
                      size=(temp_adata.n_vars,))
print(f"ct的前10个元素为{my_path[0:10]}")

ct的前10个元素为['Pathway_B' 'Pathway_B' 'Pathway_C' 'Pathway_B' 'Pathway_A' 'Pathway_C'
 'Pathway_B' 'Pathway_C' 'Pathway_C' 'Pathway_C']


In [45]:
temp_adata.var["my_pathway"] = pd.Categorical(my_path)  # Categorical的执行效率较高

# 可以看到adata.var已经包含了我们刚刚添加的注释
temp_adata.var

,my_pathway
Gene_0,Pathway_B
Gene_1,Pathway_B
Gene_2,Pathway_C
Gene_3,Pathway_B
Gene_4,Pathway_A
...,...
Gene_1995,Pathway_B
Gene_1996,Pathway_B
Gene_1997,Pathway_A
Gene_1998,Pathway_C


In [46]:
# 这时这个AnnData对象就包含了我们所添加的cell_type以及my_pathway注释：
temp_adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'
    var: 'my_pathway'

In [52]:
# 这时可以通过注释来选取数据的子集：
# 取出B cell的细胞数据
bcell_adata = temp_adata[temp_adata.obs.cell_type == "B",:]

# 取出仅包含Pathway_A的基因数据：
PA_adata = temp_adata[:,temp_adata.var.my_pathway == "Pathway_A"]

In [57]:
# 构建一个略显"复杂"的注释DataFrame
obs_meta = pd.DataFrame({
        'time_yr': np.random.choice([0, 2, 4, 8], temp_adata.n_obs),
        'subject_id': np.random.choice(['subject 1', 'subject 2', 'subject 4', 'subject 8'], temp_adata.n_obs),
        'instrument_type': np.random.choice(['type a', 'type b'], temp_adata.n_obs),
        'site': np.random.choice(['site x', 'site y'], temp_adata.n_obs),
    },
    index=temp_adata.obs.index,   # 需要与AnnData的observations相一致
)
obs_meta.head()

,time_yr,subject_id,instrument_type,site
Cell_0,2,subject 2,type b,site y
Cell_1,0,subject 2,type a,site x
Cell_2,2,subject 8,type a,site x
Cell_3,4,subject 8,type b,site x
Cell_4,4,subject 1,type b,site x


In [59]:
# 将注释添加到AnnData对象中:
new_adata = ad.AnnData(temp_adata.X, obs=obs_meta, var=temp_adata.var)

new_adata
# 可以看到多出很多注释内容

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'time_yr', 'subject_id', 'instrument_type', 'site'
    var: 'my_pathway'

3，注释矩阵管理

In [66]:
temp_adata.obsm["X_umap"] = np.random.normal(0, 1, size=(temp_adata.n_obs, 2))
temp_adata.varm["gene_stuff"] = np.random.normal(0, 1, size=(temp_adata.n_vars, 5))
temp_adata
# 可以看到X_umap与gene_stuff数据被添加

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'
    var: 'my_pathway'
    obsm: 'X_umap'
    varm: 'gene_stuff'

4，非结构性的注释信息

In [69]:
temp_adata.uns["random"] = [1, 2, 3]
temp_adata.uns

OrderedDict([('random', [1, 2, 3])])

5，Layers

In [78]:
temp_adata.layers["log_transformed"] = np.log1p(temp_adata.X)
temp_adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'cell_type'
    var: 'my_pathway'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'gene_stuff'
    layers: 'log_transformed'

In [79]:
temp_adata.layers

Layers with keys: log_transformed

In [ ]:
temp_adata.to_df(layer="log_transformed")# Layer也可以直接转换为矩阵


,Gene_0,Gene_1,Gene_2,Gene_3,Gene_4,Gene_5,Gene_6,Gene_7,Gene_8,Gene_9,...,Gene_1990,Gene_1991,Gene_1992,Gene_1993,Gene_1994,Gene_1995,Gene_1996,Gene_1997,Gene_1998,Gene_1999
Cell_0,0.693147,0.000000,1.609438,0.693147,0.000000,0.000000,0.000000,0.693147,1.386294,0.693147,...,0.000000,0.000000,0.693147,0.693147,0.000000,0.693147,0.693147,0.693147,0.693147,1.386294
Cell_1,1.609438,0.693147,0.693147,1.098612,0.693147,0.000000,0.000000,0.693147,0.693147,0.693147,...,0.000000,0.693147,0.000000,0.693147,0.693147,0.000000,0.000000,0.000000,0.693147,0.693147
Cell_2,0.000000,0.000000,0.693147,0.693147,0.693147,0.000000,0.693147,0.000000,0.693147,0.000000,...,0.000000,1.386294,0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.693147,1.098612
Cell_3,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.693147,...,0.000000,1.386294,0.693147,0.693147,0.693147,1.098612,0.000000,0.693147,0.000000,0.000000
Cell_4,1.098612,1.609438,0.000000,1.098612,1.098612,0.693147,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.098612,1.098612,1.098612,0.693147,0.693147,0.693147,0.000000,0.693147,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cell_95,0.693147,1.386294,0.000000,0.000000,1.386294,0.693147,0.693147,0.693147,1.386294,0.693147,...,0.693147,0.000000,0.693147,0.693147,0.000000,0.693147,1.098612,0.693147,0.000000,0.000000
Cell_96,0.693147,0.000000,1.098612,0.693147,0.693147,0.693147,1.386294,0.000000,0.693147,0.000000,...,1.098612,0.693147,0.000000,0.000000,1.098612,0.693147,0.693147,0.000000,1.098612,0.693147
Cell_97,1.098612,0.000000,0.693147,0.000000,1.098612,0.693147,0.693147,0.693147,0.000000,0.000000,...,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.693147,1.098612,0.000000
Cell_98,0.000000,0.693147,1.609438,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,1.098612,...,0.693147,0.000000,0.693147,0.000000,1.386294,1.791759,0.693147,0.000000,1.098612,0.000000


In [83]:
# 也可以输出为csv文件：当然前面转换为数据框之后可以直接导出为csv文件
temp_adata.to_df(layer="log_transformed").to_csv("log_transformed.csv")

6，AnnData存储与读取

In [ ]:
#保存为h5ad格式
temp_adata.write('my_results.h5ad', compression="gzip")

In [91]:
adata = ad.read('my_results.h5ad', backed='r')
adata.isbacked #此时my_results.h5ad处于open状态，即被占用的状态

/home/csn/program/miniconda3/envs/sc-python/lib/python3.12/site-packages/anndata/__init__.py:52: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


True

In [110]:
adata = ad.read_h5ad('my_results.h5ad', backed='r')
adata.isbacked #此时my_results.h5ad处于open状态，即被占用的状态，输出为True

# 此时adata可以被正常使用，且多出路径变量
adata.filename

# 当完成分析时可以解除占用：
adata.file.close() 
# 这其实就是scanpy在分析相同的数据比Seurat占用更少内存的秘诀

7，AnnData的Views与copies

In [111]:
# 在做View时候也可以使用切片：
temp_adata[:5, ['Gene_1', 'Gene_3']]

View of AnnData object with n_obs × n_vars = 5 × 2
    obs: 'cell_type'
    var: 'my_pathway'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'gene_stuff'
    layers: 'log_transformed'

In [112]:
# 而copy的方式会占据额外的内存
adata_subset = temp_adata[:5, ['Gene_1', 'Gene_3']].copy()
adata_subset

AnnData object with n_obs × n_vars = 5 × 2
    obs: 'cell_type'
    var: 'my_pathway'
    uns: 'random'
    obsm: 'X_umap'
    varm: 'gene_stuff'
    layers: 'log_transformed'